In [20]:
import os
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image

# === PARAMETERS === #
rows, cols = 5, 5
a = 1  # spacing between nodes
frame_dir = "frames_tri"
os.makedirs(frame_dir, exist_ok=True)

# === GENERATE TRIANGULAR GRID === #
def generate_triangular_grid(rows, cols, a):
    points = []
    for row in range(rows):
        for col in range(cols):
            x = col * a + (a / 2 if row % 2 else 0)
            y = row * (a * np.sqrt(3) / 2)
            points.append((x, y))
    return np.array(points)

points = generate_triangular_grid(rows, cols, a)

# === DRAW LATTICE FRAME === #
def draw_lattice(points, a, directions, filename):
    fig, ax = plt.subplots()
    ax.set_aspect('equal')
    ax.axis('off')

    point_set = set(tuple(np.round(p, 5)) for p in points)

    for x, y in points:
        for dx, dy in directions:
            nx = x + dx * a
            ny = y + dy * a
            if (round(nx, 5), round(ny, 5)) in point_set:
                ax.plot([x, nx], [y, ny], color='black', linewidth=1)

    ax.scatter(points[:, 0], points[:, 1], color='black', s=10)
    ax.set_xlim(-1, np.max(points[:, 0]) + 1)
    ax.set_ylim(-1, np.max(points[:, 1]) + 1)
    plt.tight_layout()
    plt.savefig(os.path.join(frame_dir, filename), dpi=300)
    plt.close()

# === DEFINE DIRECTION VECTORS === #
angle_deg = [0, 60, 120, 180, 240, 300]
all_dirs = [(np.cos(np.deg2rad(a)), np.sin(np.deg2rad(a))) for a in angle_deg]

# Square-style directions: horizontal (0°) and vertical (90°)
square_dirs = [(np.cos(np.deg2rad(0)), np.sin(np.deg2rad(0))),
               (np.cos(np.deg2rad(90)), np.sin(np.deg2rad(90)))]

# === GENERATE ALL 6 FRAMES === #
draw_lattice(points, a, square_dirs, "frame1_square_style.png")
draw_lattice(points, a, [all_dirs[0], all_dirs[4]], "frame2_right_down.png")
draw_lattice(points, a, [all_dirs[0], all_dirs[4], all_dirs[5]], "frame3_add_diag_300.png")
draw_lattice(points, a, all_dirs, "frame6_full_triangular.png")

# === BUILD GIF SEQUENCE === #
frame_files = [
    "frame1_square_style.png",
    "frame2_right_down.png",
    "frame3_add_diag_300.png",
    "frame6_full_triangular.png"
]

# Load images
frame_paths = [os.path.join(frame_dir, f) for f in frame_files]
frames = [Image.open(f).convert("P", dither=Image.NONE) for f in frame_paths]

# Full sequence: forward → hold → reverse
full_sequence = frames + [frames[-1]] + frames[-2::-1]

# Frame durations in ms
durations = (
    [1500, 500, 500, 500] +  # forward
    [1500] +                          # hold on final frame
    [500, 500, 500, 500]         # reverse
)

# === SAVE FINAL GIF === #
gif_output = "triangular_lattice_final.gif"
full_sequence[0].save(
    gif_output,
    save_all=True,
    append_images=full_sequence[1:],
    duration=durations,
    loop=0
)

print(f"✅ Done! GIF saved as: {gif_output}")


✅ Done! GIF saved as: triangular_lattice_final.gif
